In [1]:
%run init_notebook.py

In [64]:
import pymc as pm
import arviz as az
from src.pymc_modelling import *
import scipy
import pytensor.tensor as pt

In [3]:
## lecture 12

In [15]:
def vec_translate(a, my_dict):    
    return np.vectorize(my_dict.__getitem__)(a)

array([0.1 , 0.3 , 0.3 , 0.1 , 0.3 , 0.5 , 0.1 , 0.1 , 0.1 , 0.1 , 0.1 ,
       0.1 , 0.1 , 0.25, 0.3 , 0.3 , 0.1 , 0.3 , 0.1 , 0.3 , 0.5 , 0.1 ,
       0.25, 0.5 , 0.25, 0.25, 0.5 , 0.3 , 0.25, 0.1 , 0.1 , 0.5 , 0.25,
       0.5 , 0.1 , 0.25, 0.1 , 0.25, 0.5 , 0.1 , 0.25, 0.1 , 0.3 , 0.1 ,
       0.1 , 0.3 , 0.3 , 0.5 , 0.1 , 0.1 , 0.1 , 0.1 , 0.3 , 0.1 , 0.3 ,
       0.1 , 0.25, 0.1 , 0.1 , 0.5 , 0.25, 0.1 , 0.5 , 0.5 , 0.1 , 0.25,
       0.1 , 0.5 , 0.25, 0.1 , 0.1 , 0.1 , 0.3 , 0.3 , 0.3 , 0.3 , 0.1 ,
       0.1 , 0.5 , 0.25, 0.25, 0.1 , 0.1 , 0.25, 0.3 , 0.3 , 0.3 , 0.25,
       0.25, 0.3 , 0.25, 0.3 , 0.1 , 0.1 , 0.1 , 0.25, 0.25, 0.5 , 0.3 ,
       0.1 ])

In [31]:
col1 = np.random.randint(0, 5, 100)
col2 = np.random.randn(100)

In [54]:
X = np.array([col1, col2,], ).T

col1_beta_true = vec_translate(col1, {0: .1, 1: .1, 2: .5, 3: .3, 4: .25})
beta_true = np.array([1, .3])
sigma_true = .1
alpha_true = .2

X_true = np.array([col1_beta_true, col2],)
y_true = alpha_true + beta_true @ X_true + scipy.stats.norm(0, sigma_true).rvs(len(X)) 

In [58]:
with pm.Model() as mod1:
    
    alpha = pm.Normal('alpha', 0 , 1)
    beta_2 = pm.Normal('beta_2', 0, 1)
    
    sigma = pm.HalfNormal('sigma', 1)
    
    mu = pm.Deterministic('mu', alpha + beta_2 * X[:,1])
    
    y_ = pm.Normal('y', mu=mu, sigma=sigma, observed=y_true)
    
    prior1 = pm.sample_prior_predictive()
    idata2 = pm.sample(nuts_sampler='numpyro')
    post2 = pm.sample_posterior_predictive(idata)

Sampling: [alpha, beta_2, sigma, y]
C:\Users\LukasGrahl\miniforge3\envs\mamba_env_memoire2\Lib\site-packages\pymc\sampling\mcmc.py:273: UserWarning: Use of external NUTS sampler is still experimental
  warnings.warn("Use of external NUTS sampler is still experimental", UserWarning)


Compiling...
Compilation time =  0:00:04.565126
Sampling...


  0%|          | 0/2000 [00:00<?, ?it/s]

  0%|          | 0/2000 [00:00<?, ?it/s]

  0%|          | 0/2000 [00:00<?, ?it/s]

  0%|          | 0/2000 [00:00<?, ?it/s]

Sampling: [y]


Sampling time =  0:00:08.851166
Transforming variables...
Transformation time =  0:00:00.029522


In [ ]:
with pm.Model() as mod2:
    C = pm.MutableData('riskaversion', X[:,0].astype(int))
    
    alpha = pm.Normal('alpha', 0 , 1)
    beta_1 = pm.Normal('beta_1', 0, 1)
    beta_2 = pm.Normal('beta_2', 0, 1)
    
    sigma = pm.HalfNormal('sigma', 1)
    
    delta = pm.Dirichlet('delta', np.repeat(0.0, 4), shape=4)
    delta_j = pt.concatenate([pt.zeros(1), delta])
    delta_j_cum = pt.cumsum(delta_j)
    
    mu = pm.Deterministic('mu', alpha + beta_1 * delta_j_cum[C] +  beta_2 * X[:,1])
    
    y_ = pm.Normal('y', mu=mu, sigma=sigma, observed=y_true)
    
    prior2 = pm.sample_prior_predictive()
    idata2 = pm.sample(nuts_sampler='numpyro')
    # post2 = pm.sample_posterior_predictive(idata)

Sampling: [alpha, beta_1, beta_2, delta, sigma, y]


In [ ]:
with pm.Model() as mod_categorical:
    R = pm.MutableData('riskaversion', sub.riskaversion.factorize()[0])
    
    delta = pm.Dirichlet('delta',np.repeat(0.0, 10), shape=10)
    delta_j = pt.concatenate([pt.zeros(1), delta])
    delta_j_cum = pt.cumsum(delta_j)
    
    b_R = pm.Normal('b_risk', 0, 1, shape=1)
    
    y_hat = b_R * delta_j_cum[R]
    
    lkh = pm.Normal('y', mu=y_hat, sigma=.5, observed=sub[lst_dep_col].values[:,0])